In [1]:
import pandas as pd
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', None)
import os
import boto3

## Init Boto3 session

In [2]:
access_key_df = pd.read_csv(r'C:\Users\Christophe\Desktop\AWS\acces_key_bloc1\access_key.csv')

### According to security reasons, access key file must be stored on my own computer:
- only variables containing secret keys appears in the notebook

In [3]:
secret_id = access_key_df['Access key ID'][0]

In [4]:
secret_pass = access_key_df['Secret access key'][0]

In [5]:
session = boto3.Session(aws_access_key_id=secret_id, 
                        aws_secret_access_key=secret_pass)

In [6]:
s3 = session.resource("s3")

In [7]:
# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)

bucket-bloc1-kayak-derache


## Store Weather Data on S3 Bucket

In [8]:
path = os.getcwd()
path = str(os.path.abspath(os.path.join(path, os.pardir))) 
path = path + r'\2 - get_weather_35_destinations/weather_ready_to_sort.csv'

In [9]:
# Upload the weather data
data = open(path, 'rb')
s3.Bucket('bucket-bloc1-kayak-derache').put_object(Key='weather_ready_to_sort.csv', Body=data)

s3.Object(bucket_name='bucket-bloc1-kayak-derache', key='weather_ready_to_sort.csv')

## Store Hotels Data on S3 Bucket

In [10]:
path = os.getcwd()
path = str(os.path.abspath(os.path.join(path, os.pardir))) 
path = path + r'\3 - get_hotels_35_destinations\cleaned_and_concat_hotels\df_hotels.csv'

In [11]:
# Upload the hotels data
data = open(path, 'rb')
s3.Bucket('bucket-bloc1-kayak-derache').put_object(Key='dataframe_hotels.csv', Body=data)

s3.Object(bucket_name='bucket-bloc1-kayak-derache', key='dataframe_hotels.csv')

## Make the MERGE of both dataframes before uploading the resulting CSV file on S3 Bucket

In [13]:
weather_data = pd.read_csv('https://bucket-bloc1-kayak-derache.s3.eu-west-3.amazonaws.com/weather_ready_to_sort.csv') 
weather_data.shape


(35, 27)

In [14]:
hotels_data = pd.read_csv('https://bucket-bloc1-kayak-derache.s3.eu-west-3.amazonaws.com/dataframe_hotels.csv')
hotels_data.shape

(875, 10)

In [15]:
weather_data.head(2)

,cityname,latitude,longitude,temperature_j1,precipitation_j1,weathercode_j1,temperature_j2,precipitation_j2,weathercode_j2,temperature_j3,precipitation_j3,weathercode_j3,temperature_j4,precipitation_j4,weathercode_j4,temperature_j5,precipitation_j5,weathercode_j5,temperature_j6,precipitation_j6,weathercode_j6,temperature_j7,precipitation_j7,weathercode_j7,mean_temp,sum_precipitation,top_weather_code
0,Mont Saint Michel,48.635954,-1.511460,10.0,0.0,2,11.1,0.0,3,13.9,0.0,3,12.8,0.0,3,12.7,0.0,3,11.0,0.1,3,12.4,0.0,3,11.985714,0.1,3
1,St Malo,48.649518,-2.026041,11.4,0.0,3,11.9,0.0,3,13.4,0.0,3,12.5,0.0,3,10.5,0.0,3,10.8,0.1,3,12.2,0.0,3,11.814286,0.1,3


In [16]:
hotels_data.head(2)

,namecity,name,link,review_score,review_count,area,address,hotel_latitude,hotel_longitude,description
0,Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.fr.html?...,"8,0",4 389 expériences vécues,Le Mont-Saint-Michel,"La Caserne, 50170 Le Mont-Saint-Michel, France",48.614700,-1.509617,Vous pouvez bénéficier d'une réduction Genius ...
1,Mont Saint Michel,Mercure Mont Saint Michel,https://www.booking.com/hotel/fr/mont-saint-mi...,"8,2",3 570 expériences vécues,Le Mont-Saint-Michel,"La Caserne, 50170 Le Mont-Saint-Michel, France",48.614247,-1.510545,Installé dans des espaces verts à seulement 2 ...


In [17]:
merged_weather_hotels_data = hotels_data
merged_weather_hotels_data = merged_weather_hotels_data.merge(weather_data, left_on='namecity', right_on='cityname',how='left')

In [18]:
merged_weather_hotels_data.head(50)

,namecity,name,link,review_score,review_count,area,address,hotel_latitude,hotel_longitude,description,cityname,latitude,longitude,temperature_j1,precipitation_j1,weathercode_j1,temperature_j2,precipitation_j2,weathercode_j2,temperature_j3,precipitation_j3,weathercode_j3,temperature_j4,precipitation_j4,weathercode_j4,temperature_j5,precipitation_j5,weathercode_j5,temperature_j6,precipitation_j6,weathercode_j6,temperature_j7,precipitation_j7,weathercode_j7,mean_temp,sum_precipitation,top_weather_code
0,Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.fr.html?...,"8,0",4 389 expériences vécues,Le Mont-Saint-Michel,"La Caserne, 50170 Le Mont-Saint-Michel, France",48.614700,-1.509617,Vous pouvez bénéficier d'une réduction Genius ...,Mont Saint Michel,48.635954,-1.511460,10.0,0.0,2,11.1,0.0,3,13.9,0.0,3,12.8,0.0,3,12.7,0.0,3,11.0,0.1,3,12.4,0.0,3,11.985714,0.1,3
1,Mont Saint Michel,Mercure Mont Saint Michel,https://www.booking.com/hotel/fr/mont-saint-mi...,"8,2",3 570 expériences vécues,Le Mont-Saint-Michel,"La Caserne, 50170 Le Mont-Saint-Michel, France",48.614247,-1.510545,Installé dans des espaces verts à seulement 2 ...,Mont Saint Michel,48.635954,-1.511460,10.0,0.0,2,11.1,0.0,3,13.9,0.0,3,12.8,0.0,3,12.7,0.0,3,11.0,0.1,3,12.4,0.0,3,11.985714,0.1,3
2,Mont Saint Michel,La Mère Poulard,https://www.booking.com/hotel/fr/la-mere-poula...,"7,0",2 044 expériences vécues,Le Mont-Saint-Michel,"Grande Rue, 50170 Le Mont-Saint-Michel, France",48.635085,-1.510540,Vous pouvez bénéficier d'une réduction Genius ...,Mont Saint Michel,48.635954,-1.511460,10.0,0.0,2,11.1,0.0,3,13.9,0.0,3,12.8,0.0,3,12.7,0.0,3,11.0,0.1,3,12.4,0.0,3,11.985714,0.1,3
3,Mont Saint Michel,Hotel De La Digue,https://www.booking.com/hotel/fr/de-la-digue.f...,"7,1",2 949 expériences vécues,Le Mont-Saint-Michel,"La Caserne, 50170 Le Mont-Saint-Michel, France",48.616882,-1.510918,L'hôtel De La Digue est un établissement tradi...,Mont Saint Michel,48.635954,-1.511460,10.0,0.0,2,11.1,0.0,3,13.9,0.0,3,12.8,0.0,3,12.7,0.0,3,11.0,0.1,3,12.4,0.0,3,11.985714,0.1,3
4,Mont Saint Michel,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,"7,8",3 111 expériences vécues,Le Mont-Saint-Michel,"La Caserne, 50170 Le Mont-Saint-Michel, France",48.617587,-1.510396,Vous pouvez bénéficier d'une réduction Genius ...,Mont Saint Michel,48.635954,-1.511460,10.0,0.0,2,11.1,0.0,3,13.9,0.0,3,12.8,0.0,3,12.7,0.0,3,11.0,0.1,3,12.4,0.0,3,11.985714,0.1,3
5,Mont Saint Michel,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,"7,3",2 492 expériences vécues,Le Mont-Saint-Michel,"Grande Rue, 50170 Le Mont-Saint-Michel, France",48.635349,-1.510379,Vous pouvez bénéficier d'une réduction Genius ...,Mont Saint Michel,48.635954,-1.511460,10.0,0.0,2,11.1,0.0,3,13.9,0.0,3,12.8,0.0,3,12.7,0.0,3,11.0,0.1,3,12.4,0.0,3,11.985714,0.1,3
6,Mont Saint Michel,Le Mouton Blanc,https://www.booking.com/hotel/fr/le-mouton-bla...,"7,0",1 578 expériences vécues,Le Mont-Saint-Michel,"Grande Rue , 50170 Le Mont-Saint-Michel, France",48.636023,-1.509896,Vous pouvez bénéficier d'une réduction Genius ...,Mont Saint Michel,48.635954,-1.511460,10.0,0.0,2,11.1,0.0,3,13.9,0.0,3,12.8,0.0,3,12.7,0.0,3,11.0,0.1,3,12.4,0.0,3,11.985714,0.1,3
7,Mont Saint Michel,La Vieille Auberge,https://www.booking.com/hotel/fr/la-vieille-au...,"7,1",1 118 expériences vécues,Le Mont-Saint-Michel,"Grande Rue, 50170 Le Mont-Saint-Michel, France",48.636063,-1.511457,La Vieille Auberge vous accueille dans le vill...,Mont Saint Michel,48.635954,-1.511460,10.0,0.0,2,11.1,0.0,3,13.9,0.0,3,12.8,0.0,3,12.7,0.0,3,11.0,0.1,3,12.4,0.0,3,11.985714,0.1,3
8,Mont Saint Michel,Hotel Gabriel,https://www.booking.com/hotel/fr/hotel-gabriel...,"7,9",2 427 expériences vécues,Le Mont-Saint-Michel,"Route du Mont Saint Michel, 50170 Le Mont-Sain...",48.615381,-1.510710,Vous pouvez bénéficier d'une réduction Genius ...,Mont Saint Michel,48.635954,-1.511460,10.0,0.0,2,11.1,0.0,3,13.9,0.0,3,12.8,0.0,3,12.7,0.0,3,11.0,0.1,3,12.4,0

## Save this new Dataframe

In [19]:
merged_weather_hotels_data.to_csv('merged_weather_hotels_data.csv', index=False)

## Upload on my S3 Bucket

In [20]:
# Upload the hotels + weather merged data
data = open('merged_weather_hotels_data.csv', 'rb')
s3.Bucket('bucket-bloc1-kayak-derache').put_object(Key='merged_weather_hotels_data.csv', Body=data)

s3.Object(bucket_name='bucket-bloc1-kayak-derache', key='merged_weather_hotels_data.csv')